In [24]:
from pyspark.sql import SparkSession, functions as F, types as T
from geopy.distance import geodesic
from pathlib import Path

DATA_DIR = Path.cwd().parent / "data"     
trips_csv    = DATA_DIR / "trips.csv"
stations_csv = DATA_DIR / "stations.csv"

# Инициализация SparkSession

spark = (
    SparkSession.builder
        .appName("BikeShare-Inline-Output")
        .master("local[4]")
        .config("spark.executor.memory", "2g")
        .config("spark.driver.memory",   "2g")
        .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")



In [25]:
# Читаем CSV в DataFrame’ы

trip_schema = T.StructType([
    T.StructField("trip_id",            T.IntegerType(),  False),
    T.StructField("duration",           T.IntegerType(),  False),
    T.StructField("start_date",         T.TimestampType(),False),
    T.StructField("start_station_name", T.StringType(),   False),
    T.StructField("start_station_id",   T.IntegerType(),  False),
    T.StructField("end_date",           T.TimestampType(),False),
    T.StructField("end_station_name",   T.StringType(),   False),
    T.StructField("end_station_id",     T.IntegerType(),  False),
    T.StructField("bike_id",            T.IntegerType(),  False),
    T.StructField("subscription_type",  T.StringType(),   True),
    T.StructField("zip_code",           T.StringType(),   True)
])

station_schema = T.StructType([
    T.StructField("station_id",  T.IntegerType(), False),
    T.StructField("name",        T.StringType(),  False),
    T.StructField("lat",         T.DoubleType(),  False),
    T.StructField("long",        T.DoubleType(),  False),
    T.StructField("dockcount",   T.IntegerType(), False),
    T.StructField("landmark",    T.StringType(),  True),
    T.StructField("installation",T.DateType(),    False)
])

trips_df    = spark.read.csv(str(trips_csv),    schema=trip_schema, header=True).cache()
stations_df = spark.read.csv(str(stations_csv), schema=station_schema, header=True).cache()

1.	Найти велосипед с максимальным временем пробега.

In [26]:
top_bike_row = (
    trips_df.groupBy("bike_id")
            .agg(F.sum("duration").alias("total_duration"))
            .orderBy(F.desc("total_duration"))
            .first()
)
top_bike_id, top_bike_duration = top_bike_row.bike_id, top_bike_row.total_duration
print(f"[1] Bike with max ride time → id = {top_bike_id}, total = {top_bike_duration:,} sec")

[1] Bike with max ride time → id = 535, total = 18,611,693 sec


2.	Найти наибольшее геодезическое расстояние между станциями.

In [27]:
R_EARTH_KM = 6371.0088   # радиус Земли WGS-84

s1 = stations_df.select(
        F.col("station_id").alias("id1"),
        F.radians("lat").alias("lat1_rad"),
        F.radians("long").alias("lon1_rad")
)
s2 = stations_df.select(
        F.col("station_id").alias("id2"),
        F.radians("lat").alias("lat2_rad"),
        F.radians("long").alias("lon2_rad")
)

# формула гаверсинуса: 2*R*asin( sqrt( sin²(Δφ/2)+cos φ1 cos φ2 sin²(Δλ/2) ) )
delta_lat = F.col("lat2_rad") - F.col("lat1_rad")
delta_lon = F.col("lon2_rad") - F.col("lon1_rad")

a = (F.sin(delta_lat / 2) ** 2 +
     F.cos("lat1_rad") * F.cos("lat2_rad") * F.sin(delta_lon / 2) ** 2)

dist_expr = F.lit(2 * R_EARTH_KM) * F.asin(F.sqrt(a))

max_row = (
    s1.crossJoin(s2)
      .where(F.col("id1") < F.col("id2"))      # убираем дубликаты и пары сам-с-собой
      .withColumn("dist_km", dist_expr)
      .orderBy(F.desc("dist_km"))
      .select("id1", "id2", "dist_km")
      .first()
)

print(f"[2] Longest station distance → {max_row.dist_km:,.2f} km "
      f"(stations {max_row.id1} ↔ {max_row.id2})")

[2] Longest station distance → 69.92 km (stations 16 ↔ 60)


3.	Найти путь велосипеда с максимальным временем пробега через станции.

In [28]:
path_rows = (
    trips_df.where(F.col("bike_id") == top_bike_id)
            .select("start_date", "start_station_name", "end_station_name")
            .orderBy("start_date")
            .collect()
)
station_path = [r.start_station_name for r in path_rows] + [path_rows[-1].end_station_name]
print(f"[3] Path of bike {top_bike_id} → " + " → ".join(station_path))

[3] Path of bike 535 → San Francisco Caltrain (Townsend at 4th) → Post at Kearney → Market at Sansome → San Francisco Caltrain 2 (330 Townsend) → 2nd at Townsend → San Francisco City Hall → Civic Center BART (7th at Market) → Post at Kearney → Embarcadero at Sansome → Washington at Kearney → Market at Sansome → Market at Sansome → 2nd at Folsom → Temporary Transbay Terminal (Howard at Beale) → 2nd at Townsend → Embarcadero at Sansome → Clay at Battery → Harry Bridges Plaza (Ferry Building) → Clay at Battery → San Francisco Caltrain (Townsend at 4th) → Steuart at Market → 2nd at Townsend → Harry Bridges Plaza (Ferry Building) → Townsend at 7th → San Francisco Caltrain 2 (330 Townsend) → San Francisco Caltrain 2 (330 Townsend) → Steuart at Market → San Francisco Caltrain (Townsend at 4th) → 2nd at South Park → Post at Kearney → 2nd at Folsom → Mechanics Plaza (Market at Battery) → Powell at Post (Union Square) → Powell at Post (Union Square) → Powell at Post (Union Square) → Mechanics Pl

4.	Найти количество велосипедов в системе.

In [29]:
bike_count = trips_df.select("bike_id").distinct().count()
print(f"[4] Total bikes in system → {bike_count}")

[4] Total bikes in system → 700


5.	Найти пользователей потративших на поездки более 3 часов.

In [30]:
heavy_users_rows = (
    trips_df.groupBy("zip_code")
            .agg(F.sum("duration").alias("total_duration"))
            .where( (F.col("total_duration") > 3 * 60 * 60) &
                    F.col("zip_code").isNotNull() )
            .select("zip_code")
            .collect()            # ← всё ещё одна Spark-операция, но без Python-UDF
)

heavy_users = [r.zip_code for r in heavy_users_rows]

print(f"[5] Users riding >3 h → "
      f"{', '.join(heavy_users) if heavy_users else '— none —'}")

spark.stop()

[5] Users riding >3 h → 94102, 95134, 84606, 80305, 60070, 95519, 43085, 91910, 77339, 48063, 95138, 94610, 94404, 80301, 91326, 90742, 11205, 95212, 94309, 22314, 23113, 55443, 97239, 94592, 7650, 11106, 90211, 93013, 30324, 14000, 94966, 84102, 95834, 44500, 10250, 94568, 94015, 95015, 91780, 60661, 4665, 31005, 27701, 3121, 6820, 94079, 10110, 7, 28034, 95130, 75219, 46614, 11218, 81611, 94599, 6033, 45219, 5052, 97330, 53714, 85251, 94550, 77098, 6605, 19333, 33805, 2129, 76148, 84098, 42071, 91766, 94107, 95020, 80401, 76039, 81377, 92879, 97206, 91331, 55413, 33129, 7015, 2200, 94068, 2144, 94621, 90802, 60611, 92116, 70119, 10512, 90745, 93306, 87120, 90042, 1207, 77054, 83843, 85254, 91001, 2016, 94070, 80435, 75070, 94534, 92129, 11237, 48130, 80218, 95959, 63103, 92058, 49401, 60486, 6300, 94062, 2138, 91304, 90026, 6245, 2115, 98144, 2012, 89512, 11021, 94034, 20024, 87111, 60048, 66044, 90503, 7652, 93906, 21228, 98109, 90029, 27516, 23223, 94561, 91301, 95454, 96708, 2139,